In [1]:
import datetime
import os 
from dotenv import load_dotenv
import numpy as np

path = '/home/rafaelfabrichimidt/Documentos/projetos/env_vars/'
envfile = '.env'

load_dotenv(dotenv_path = path + envfile)

True

In [3]:
import duckdb
conn = duckdb.execute("ATTACH 'dbname=postgresql_oltp user=admin host=localhost, password=12345' AS db (TYPE POSTGRES); ")

In [26]:
df_insc = conn.sql('''
                 SELECT nu_inscricao, 
                        nu_nota_comp1, 
                        nu_nota_comp2, 
                        nu_nota_comp3, 
                        nu_nota_comp4,
                        nu_nota_comp5,
                        nu_nota_redacao 
                 FROM db.enem.base_enem
                 WHERE nu_nota_redacao IS NOT NULL
                 LIMIT 100''')

df_insc.df().to_csv('Experimento_1.csv', index = False)

In [38]:
alunos_exp_1 = pd.read_csv('Experimento_1.csv')

In [4]:
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

In [5]:
from openai import OpenAI
client = OpenAI()

In [11]:
system_prompt_sql = 'Você é responsável por construir queries SQL Postgres, para a query SQL funcionar deve-se adicionar no FROM database.schema.table. Retorne apenas a query SQL'

descricao_base_enem_sql = '''
                                database=db;
                                schema=enem;
                                table=base_enem;
                                campo no_municipio_esc tipo varchar Nome do município da escola;
                                campo sg_uf_esc tipo varchar Sigla da Unidade da Federação da escola;
                                campo nu_inscricao tipo int Número da inscrição do aluno; 
                                campo nu_nota_cn tipo int Nota da prova de Ciências da Natureza; 
                                campo nu_nota_ch tipo int Nota da prova de Ciências Humanas;
                                campo nu_nota_lc tipo int Nota da prova de Linguagens e Códigos;
                                campo nu_nota_mt tipo int Nota da Nota da prova de Matemática;
                                campo tp_status_redacao tipo int Situação da redação do participante;
                                campo nu_nota_comp1 tipo int Nota da competência 1 da prova de redação; 
                                campo nu_nota_comp2 tipo int Nota da competência 2 da prova de redação; 
                                campo nu_nota_comp3 tipo int Nota da competência 3 da prova de redação;
                                campo nu_nota_comp4 tipo int Nota da competência 4 da prova de redação; 
                                campo nu_nota_comp5 tipo int Nota da competência 5 da prova de redação; 
                                campo nu_nota_redacao tipo int Nota final da prova de redação ;

'''

user_mensage_question = 'Sou o aluno com inscrição 210061103945 desejava saber meu desempenho em na Redação do Enem e como posso melhorar no próximo ano?'



In [12]:
mensagens = [
    {'role': 'system', 'content': system_prompt_sql},
    {'role': 'system', 'content': descricao_base_enem_sql},
    {'role': 'user', 'content': user_mensage_question}
]

In [13]:
resposta = client.chat.completions.create(
    model='gpt-4o-mini',
    messages=mensagens,
    max_tokens=200,
    temperature=0.7,
)

mensagem_resp = resposta.choices[0].message.content
print(mensagem_resp)

sql_query = mensagem_resp.replace("```", "").split('sql')[1]

```sql
SELECT nu_nota_redacao, 
       nu_nota_comp1, 
       nu_nota_comp2, 
       nu_nota_comp3, 
       nu_nota_comp4, 
       nu_nota_comp5 
FROM db.enem.base_enem 
WHERE nu_inscricao = 210061103945;
```


In [14]:
df = conn.sql(sql_query)

In [16]:
df.fetchnumpy()

{'nu_nota_redacao': array([700.]),
 'nu_nota_comp1': array([140.]),
 'nu_nota_comp2': array([200.]),
 'nu_nota_comp3': array([100.]),
 'nu_nota_comp4': array([120.]),
 'nu_nota_comp5': array([140.])}

In [21]:
system_prompt_dialogue = '''
                                Você é um professor responsável por analisar o desempenho de 
                                um aluno no Enem e recomendar estudos de acordo com sua pontuação
                                um resumo das Competência de cada área, não precisa repetir as 
                                competências apenas descrever e sugerir exemplos de melhoria em cada competência
'''


descricao_detalhada_enem = open(file = 'descricao_detalhada.txt', mode = 'r').read()

user_mensage_with_data = user_mensage_question + 'Os dados com detalhes do desempenho do Aluno encontra-se abaixo \n ' + f' {df.fetchnumpy()} Ajude o aluno para melhorar sua nota: '

In [22]:
prompt_resposta = [
    {'role': 'system', 'content': system_prompt_dialogue},
    {'role': 'system', 'content': descricao_detalhada_enem},                                
    {'role': 'user', 'content': user_mensage_with_data}
     ]

In [23]:
resposta = client.chat.completions.create(
    model='gpt-4o-mini',
    messages=prompt_resposta,
    max_tokens=400,
    temperature=0.7,
)

mensagem_resp = resposta.choices[0].message.content
print(mensagem_resp)


Caro aluno,

Vamos analisar seu desempenho na redação do Enem, com base nas notas que você obteve em cada uma das competências. Sua nota final foi 700 pontos, o que indica um desempenho muito bom, mas ainda há áreas para melhorar. 

### Desempenho por Competência:

1. **Competência 1 (Domínio da modalidade escrita formal da Língua Portuguesa)**: Nota 140
   - Você demonstrou um bom domínio da escrita formal, mas há espaço para aprimoramento. Para melhorar, pratique a escrita de textos, focando em gramática e ortografia. Leia mais obras literárias e jornais para absorver diferentes estilos de escrita.

2. **Competência 2 (Compreensão da proposta de redação)**: Nota 200
   - Excelente desempenho! Você conseguiu compreender plenamente a proposta e aplicar conceitos adequados. Continue praticando a estruturação de textos dissertativo-argumentativos para manter essa competência.

3. **Competência 3 (Seleção e organização de informações)**: Nota 100
   - Aqui está uma área que precisa de mai